In [1]:
import pandas as pd
from model import Poly2SLS, PredPolyRidge, PolySLSRidge, LinearModel, MedianHeuristic, PvalueLog, ConfIntHSIC
import patsy
import numpy as np
from functools import partial
from statsmodels.sandbox.regression.gmm import LinearIVGMM
from statsmodels.formula.api import ols
import itertools
from joblib import Parallel, delayed
from kernel import PTKGauss, CategoryKernel
from torch.utils.data import TensorDataset
import torch
from utils import *

In [31]:
df = pd.read_csv("QOB", sep=' ')
rename_dict = {'v1': 'AGE'
,'v2': 'AGEQ'
,'v4': 'EDUC'
,'v5': 'ENOCENT'
,'v6': 'ESOCENT'
,'v9': 'LWKLYWGE'
,'v10': 'MARRIED'
,'v11': 'MIDATL'
,'v12': 'MT'
,'v13': 'NEWENG'
,'v16': 'CENSUS'
,'v18': 'QOB'
,'v19': 'RACE'
,'v20': 'SMSA'
,'v21': 'SOATL'
,'v24': 'WNOCENT'
,'v25': 'WSOCENT'
,'v27': 'YOB'}

df = df.rename(rename_dict, axis=1)

# only men between 40-49
df = df.query("AGE >= 40 and AGE <= 49")
# create dummies for YOB and QOB
df['Q4'] = (df['QOB'] == 4).astype(int)
df = pd.get_dummies(df, columns=['YOB', 'QOB'], drop_first=False)

In [32]:
f_YOB = [c for c in df.columns if 'YOB_' in c]
f_QOB = [c for c in df.columns if 'QOB_' in c]
f_Z = [c1 + ':' + c2 for c1 in f_YOB for c2 in f_QOB]

# X = patsy.dmatrix("~ 1 + EDUC +" + str.join('+', f_YOB), data=df, return_type='matrix')
X = patsy.dmatrix("~ 1 + EDUC", data=df, return_type='matrix')
Y = df['LWKLYWGE'].to_numpy()
# Z = patsy.dmatrix("~ 0 + C(YOB)*C(QOB)", data=df, return_type='matrix')
# Z = patsy.dmatrix("~ -1 +" + str.join('+', f_Z), data=df, return_type='matrix')
Z = patsy.dmatrix("~ 1 + Q4", data=df, return_type='matrix')

In [33]:
OLS = PredPolyRidge(degree=1, bias=False, alpha=0.0)
GMM = LinearIVGMM(Y, X, Z)
# GMM = PolySLSRidge(degree=1,bias=False, alpha=0)

In [34]:
OLS.fit(X, Y)
gmm_res = GMM.fit()

In [35]:
OLS.reg.coef_[1]

0.07099979089901186

In [36]:
gmm_res.params[1]

0.11412747754366137

In [37]:
gmm_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             LinearIVGMM Results                              
==============================================================================
Dep. Variable:                      y   Hansen J:                    1.655e-14
Model:                    LinearIVGMM   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Thu, 20 Jan 2022                                         
Time:                        23:33:19                                         
No. Observations:              330450                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4398      0.510      8.707      0.000       3.440       5.439
x1             0.1141      0.040      2.862      0.004       0.036       0.192
==============================================================================
"""

In [66]:
def sample(X, Y, Z, m):
    idx = np.random.choice(X.shape[0], m, replace=False)
    return np.asarray(X[idx]), np.asarray(Y[idx]), np.asarray(Z[idx])

In [115]:
def invert_test(p, X, Y, Z, kernels, method='gamma'):
    res = Y - p * X
    res = dhsic_test(res, Z, kernels, 
                     statistics=True, method=method, B=100)
    return res

dHSIC = None

def worker_init():
    from rpy2.robjects import numpy2ri
    numpy2ri.activate()
    global dHSIC
    dHSIC = packages.importr("dHSIC")
    
def applyParallel(X, Y, Z, kernels, invert_test, param_range):
    func = partial(invert_test, X=X,Y=Y,Z=Z, kernels=kernels)
    from multiprocessing import Pool, cpu_count
    with Pool(cpu_count(), worker_init) as p:
        ret_list = p.map(func, param_range)
    return ret_list

In [122]:
invert_test(0.09, X_m, Y_m, Z_m, kernels)

(0.6070179403043305, 0.13681084583438974)

In [110]:
# hsic_conf = ConfIntHSIC(kernels=('gaussian', 'discrete'))
m = 5000
X_m, Y_m, Z_m = sample(X[:, 1], Y, Z[:, 1], m)
param_range = np.linspace(0.05, 0.15, 10)
kernels = ['gaussian', 'discrete']

applyParallel(X_m,Y_m,Z_m,kernels,invert_test,param_range)

[(0.495074535190045, 0.17721865170350348),
 (0.6047978015625642, 0.14187366076112706),
 (0.6578220947081921, 0.12614963393664524),
 (0.6412798184094335, 0.13065520873267733),
 (0.583149828031816, 0.14750903895355272),
 (0.5191772181354357, 0.16754917978978323),
 (0.468132347605381, 0.18461441498973663),
 (0.43445577172347516, 0.19646363787706456),
 (0.4173658687541429, 0.20295506302114852),
 (0.42074194877563875, 0.20110366465042606)]

In [25]:
se_callback = MedianHeuristic()
pval_callback = PvalueLog(B=50)

hsic_net = fit_restart(trainloader, hsic_net, pval_callback, 5, se_callback, num_restart=5)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type       | Params
---------------------------------------
0 | MSE     | MSELoss    | 0     
1 | kernels | Kernels    | 0     
2 | layers  | Sequential | 13    
---------------------------------------
13        Trainable params
0         Non-trainable params
13        Total params
0.000     Total estimated model params size (MB)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

R[write to console]: Error: cannot allocate vector of size 813.6 Gb



RRuntimeError: Error: cannot allocate vector of size 813.6 Gb


In [26]:
hsic_net.layers[0].weight

Parameter containing:
tensor([[ 0.7419,  0.0606,  0.0033,  0.0088, -0.0121, -0.0158, -0.0129, -0.0287,
         -0.0419, -0.0322, -0.0411, -0.0550, -0.0513]], requires_grad=True)